In [1]:
version = 10290  # eps = 0.001
# version = 9259  # eps = 0.1
version =74649  # k = 3
# version = 11566  # k = 4
# version = 21622 # M
# version = 72646 # k = 5
# version = 93703 # M, k=3
# version = 56780 # M, k=4
version = 24880

In [2]:
import torch
import numpy as np
import spock_reg_model

/Users/simon/mambaforge/lib/python3.10/site-packages/juliacall/__init__.py:60: UserWarning: torch was imported before juliacall. This may cause a segfault. To avoid this, import juliacall before importing torch. For updates, see https://github.com/pytorch/pytorch/issues/78829.
  warnings.warn(


[juliapkg] Found dependencies: /Users/simon/mambaforge/lib/python3.10/site-packages/juliacall/juliapkg.json
[juliapkg] Found dependencies: /Users/simon/mambaforge/lib/python3.10/site-packages/pysr/juliapkg.json
[juliapkg] Found dependencies: /Users/simon/mambaforge/lib/python3.10/site-packages/juliapkg/juliapkg.json
[juliapkg] Locating Julia ~1.6.7, ~1.7, ~1.8, ~1.9, =1.10.0
[juliapkg] Querying Julia versions from https://julialang-s3.julialang.org/bin/versions.json
[juliapkg] WARNING: About to install Julia 1.10.0 to /usr/local/anaconda3/julia_env/pyjuliapkg/install.
[juliapkg]   If you use juliapkg in more than one environment, you are likely to
[juliapkg]   have Julia installed in multiple locations. It is recommended to
[juliapkg]   install JuliaUp (https://github.com/JuliaLang/juliaup) or Julia
[juliapkg]   (https://julialang.org/downloads) yourself.
[juliapkg] Downloading Julia from https://julialang-s3.julialang.org/bin/mac/aarch64/1.10/julia-1.10.0-macaarch64.dmg
             d

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed JLLWrappers ────────── v1.6.1
   Installed Adapt ──────────────── v4.1.1
   Installed SpecialFunctions ───── v2.5.0
   Installed ArrayInterface ─────── v7.18.0
   Installed JSON3 ──────────────── v1.14.1
   Installed PtrArrays ──────────── v1.2.1
   Installed Tables ─────────────── v1.12.0
   Installed Tricks ─────────────── v0.1.9
   Installed ConstructionBase ───── v1.5.8
   Installed LogExpFunctions ────── v0.3.29
   Installed CommonSubexpressions ─ v0.3.1
   Installed OrderedCollections ─── v1.7.0
   Installed ForwardDiff ────────── v0.10.38
   Installed FillArrays ─────────── v1.13.0
   Installed StatsBase ──────────── v0.34.4
   Installed Compat ─────────────── v4.16.0
   Installed AliasTables ────────── v1.1.3
   Installed LineSearches ───────── v7.3.0
   Installed CondaPkg ───────────── v0.2.24
   Installed FiniteDiff ─────────── v2.26.2
   Installed StructTypes ────────── 

Detected Jupyter notebook. Loading juliacall extension. Set `PYSR_AUTOLOAD_EXTENSIONS=no` to disable.


In [1]:
labels = ['time', 'e+_near', 'e-_near', 'max_strength_mmr_near', 'e+_far', 'e-_far', 'max_strength_mmr_far', 'megno', 'a1', 'e1', 'i1', 'cos_Omega1', 'sin_Omega1', 'cos_pomega1', 'sin_pomega1', 'cos_theta1', 'sin_theta1', 'a2', 'e2', 'i2', 'cos_Omega2', 'sin_Omega2', 'cos_pomega2', 'sin_pomega2', 'cos_theta2', 'sin_theta2', 'a3', 'e3', 'i3', 'cos_Omega3', 'sin_Omega3', 'cos_pomega3', 'sin_pomega3', 'cos_theta3', 'sin_theta3', 'm1', 'm2', 'm3', 'nan_mmr_near', 'nan_mmr_far', 'nan_megno']

# not all of these labels are actually used. for training, these inputs are zeroed out, but still passed in as zeroes.
# ideally, the linear layer ignores them, which does happen if i do l1 regularization to it
skipped = ['nan_mmr_near', 'nan_mmr_far', 'nan_megno', 'e+_near', 'e-_near', 'max_strength_mmr_near', 'e+_far', 'e-_far', 'max_strength_mmr_far', 'megno']

In [2]:
len(labels)

41

In [5]:
def convert_to_latex(label):
    # 1. if it ends in a number, add an underscore
    if label[-1].isdigit():
        label = label[:-1] + '_' + label[-1]
    # 2. replace sin/cos with \sin/\cos
    label = label.replace('sin', '\\sin')
    label = label.replace('cos', '\\cos')
    label = label.replace('_Omega', '\\Omega')
    label = label.replace('_pomega', '\\omega')
    label = label.replace('_theta', '\\theta')
    return label

latex_labels = [convert_to_latex(label) for label in labels]
print(latex_labels)

['time', 'e+_near', 'e-_near', 'max_strength_mmr_near', 'e+_far', 'e-_far', 'max_strength_mmr_far', 'megno', 'a_1', 'e_1', 'i_1', '\\cos\\Omega_1', '\\sin\\Omega_1', '\\cos\\omega_1', '\\sin\\omega_1', '\\cos\\theta_1', '\\sin\\theta_1', 'a_2', 'e_2', 'i_2', '\\cos\\Omega_2', '\\sin\\Omega_2', '\\cos\\omega_2', '\\sin\\omega_2', '\\cos\\theta_2', '\\sin\\theta_2', 'a_3', 'e_3', 'i_3', '\\cos\\Omega_3', '\\sin\\Omega_3', '\\cos\\omega_3', '\\sin\\omega_3', '\\cos\\theta_3', '\\sin\\theta_3', 'm_1', 'm_2', 'm_3', 'nan_mmr_near', 'nan_mmr_far', 'nan_megno']


In [6]:
from sklearn.preprocessing import StandardScaler
import numpy as np
ssX = StandardScaler()
ssX.scale_ = np.array([2.88976974e+03, 6.10019661e-02, 4.03849732e-02, 4.81638693e+01,
           6.72583662e-02, 4.17939679e-02, 8.15995339e+00, 2.26871589e+01,
           4.73612029e-03, 7.09223721e-02, 3.06455099e-02, 7.10726478e-01,
           7.03392022e-01, 7.07873597e-01, 7.06030923e-01, 7.04728204e-01,
           7.09420909e-01, 1.90740659e-01, 4.75502285e-02, 2.77188320e-02,
           7.08891412e-01, 7.05214134e-01, 7.09786887e-01, 7.04371833e-01,
           7.04371110e-01, 7.09828420e-01, 3.33589977e-01, 5.20857790e-02,
           2.84763136e-02, 7.02210626e-01, 7.11815232e-01, 7.10512240e-01,
           7.03646004e-01, 7.08017286e-01, 7.06162814e-01, 2.12569430e-05,
           2.35019125e-05, 2.04211110e-05, 7.51048890e-02, 3.94254400e-01,
           7.11351099e-02])
ssX.mean_ = np.array([ 4.95458585e+03,  5.67411891e-02,  3.83176945e-02,  2.97223474e+00,
           6.29733979e-02,  3.50074471e-02,  6.72845676e-01,  9.92794768e+00,
           9.99628430e-01,  5.39591547e-02,  2.92795061e-02,  2.12480714e-03,
          -1.01500319e-02,  1.82667162e-02,  1.00813201e-02,  5.74404197e-03,
           6.86570242e-03,  1.25316320e+00,  4.76946516e-02,  2.71326280e-02,
           7.02054326e-03,  9.83378673e-03, -5.70616748e-03,  5.50782881e-03,
          -8.44213953e-04,  2.05958338e-03,  1.57866569e+00,  4.31476211e-02,
           2.73316392e-02,  1.05505555e-02,  1.03922250e-02,  7.36865006e-03,
          -6.00523246e-04,  6.53016990e-03, -1.72038113e-03,  1.24807860e-05,
           1.60314173e-05,  1.21732696e-05,  5.67292645e-03,  1.92488263e-01,
           5.08607199e-03])
ssX.var_ = ssX.scale_**2

In [7]:
# load when on the cluster
model = spock_reg_model.load(version)
feature_nn = model.feature_nn

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [8]:
# if model.hparams['combined_mass_feature']:
#     print('adding combined mass label')
#     labels.append('(m1+m2+m3)')

In [9]:
# load when local
# feature_nn = torch.load(f'models/{version}_feature_nn.pt')

In [10]:
input_linear = feature_nn.linear.weight * feature_nn.mask
input_linear = input_linear.detach().numpy()
if feature_nn.linear.bias is not None:
    input_bias = feature_nn.linear.bias.detach().numpy()
else:
    input_bias = np.zeros(input_linear.shape[0])

NameError: name 'feature_nn' is not defined

In [11]:
# m_i is the mean of the i'th feature, s_i is the standard deviation
# get the linear transformation that creates feature i
def linear_transformation(i):
    return input_linear[i]

In [12]:
# let's make the linear transformation a bit easier to read
def format_num(x, latex=False):
    if abs(x) > 1000:
        x2 = 100 * (x // 100)
        return str(x2)
    # if abs(x) > 10:
        # return f'{x:.0f}'
    # if abs(x) > 1:
        # return f'{x:.2f}'
    # if abs(x) > 0.1:
        # return f'{x:.2f}'
    # if abs(x) > 0.01:
        # return f'{x:.3f}'
    # elif abs(x) > 0.001:
        # return f'{x:.4f}'
    else:
        return f'{x:.3g}'

format_vec = np.vectorize(format_num)

In [13]:
import sympy as sp
sym_vars = {lbl: sp.Symbol(lbl, real=True) for lbl in labels}

def simplify_scaled_feature(transformation, bias=0):
    # Create symbolic variables for each feature

    expr = bias

    # Add each transformed feature (unscaled)
    for f_idx in range(len(labels)):
        c = transformation[f_idx]
        if c != 0:
            label = labels[f_idx]
            mean_j = ssX.mean_[f_idx]
            scale_j = ssX.scale_[f_idx]
            expr += c * (sym_vars[label] - mean_j) / scale_j

    expr = sp.simplify(expr)
    return expr

In [14]:
def format_sympy_expr(expr, latex=False):
    # replace labels with latex labels (change character from labels[i] to latex_labels[i])
    if latex:
        for lbl, sym in sym_vars.items():
            i = labels.index(lbl)
            new_lbl = latex_labels[i]
            expr = expr.subs(sym, sp.Symbol(new_lbl, real=True))

    coeffs = expr.as_coefficients_dict()

    terms_str = []
    const_str = None
    for var, coef in coeffs.items():
        if var == 1:
            const_str = format_num(coef, latex)
        else:
            times = '' if latex else '*'
            terms_str.append(f'{format_num(coef, latex)} {times} {var}')

    if const_str is not None:
        terms_str.append(const_str)

    return ' + '.join(terms_str)

In [15]:
def format_transformation(transformation, bias, latex):
    sorted_ixs = np.argsort(np.abs(transformation))[::-1]
    times = '' if latex else '*'
    used_labels = latex_labels if latex else labels
    features = [f'{format_num(transformation[i], latex)} {times} {used_labels[i]}' for i in sorted_ixs if transformation[i] != 0]
    if bias != 0:
        features = [format_num(bias, latex)] + features
    return ' + '.join(features)

In [16]:
expr = simplify_scaled_feature(linear_transformation(0), bias=0)

NameError: name 'input_linear' is not defined

In [17]:
def get_scaled_feature_bias(i):
    transformation = linear_transformation(i)
    bias = input_bias[i]
    expr = simplify_scaled_feature(transformation, bias)
    return expr.as_coefficients_dict().get(1, 0)

In [18]:
# load pysr f2 equations
import pickle
reg = pickle.load(open('sr_results/11003.pkl', 'rb'))
results = reg.equations_[0]

In [19]:
entry = results.iloc[8]

In [20]:
expr = entry.sympy_format

In [21]:
def add_bias_to_mean_terms(expr):
    replacements = {}
    for symbol in expr.free_symbols:
        if symbol.name.startswith('m') and symbol.name[1:].isdigit():
            i = symbol.name[1:]  # get the number after 'm'
            replacements[symbol] = symbol + get_scaled_feature_bias(int(i))
    return expr.xreplace(replacements)

In [22]:
results['sympy_format'] = results['sympy_format'].apply(add_bias_to_mean_terms)

NameError: name 'input_linear' is not defined

In [36]:
results

,complexity,loss,score,equation,sympy_format,lambda_format
0,1,5.653270,0.000000,7.0051637,7.00516370000000,PySRFunction(X=>7.00516370000000)
1,3,4.978444,0.063558,(6.990491 - m2),0.98404180377183 - m2,PySRFunction(X=>6.990491 - m2)
2,4,4.918939,0.012024,(7.1660094 - sin(m2)),7.1660094 - sin(m2 + 6.00644919622817),PySRFunction(X=>7.1660094 - sin(m2))
3,5,4.030765,0.199137,((s4 ^ -0.15489304) * 3.6422653),3.6422653/s4**0.15489304,PySRFunction(X=>3.6422653/s4**0.15489304)
4,7,3.305422,0.099196,(((s4 ^ -0.15921338) * 3.5656016) - m2),-m2 + 3.5656016/s4**0.15921338 - 6.00644919622817,PySRFunction(X=>-m2 + 3.5656016/s4**0.15921338)
5,9,3.068312,0.037218,((((s4 * s1) ^ -0.15921338) + 3.5656016) - m2),-m2 + (s1*s4)**(-0.15921338) - 2.44084759622817,PySRFunction(X=>-m2 + (s1*s4)**(-0.15921338) +...
6,11,2.696698,0.064550,((((s4 ^ -0.30986544) - m2) * (0.14308752 ^ s1...,0.14308752**s1*(-m2 + s4**(-0.30986544) - 6.00...,PySRFunction(X=>0.14308752**s1*(-m2 + s4**(-0....
7,13,2.646566,0.009383,(((((s4 ^ -0.31699488) - s6) - m2) * (0.160041...,0.16004194**s1*(-m2 + s4**(-0.31699488) - s6 -...,PySRFunction(X=>0.16004194**s1*(-m2 + s4**(-0....
8,14,2.478105,0.065769,((((s4 ^ -0.32957777) - (sin(m2) - m7)) * (0.0...,0.058552526**s1*(m7 + s4**(-0.32957777) - sin(...,PySRFunction(X=>0.058552526**s1*(m7 + s4**(-0....
9,15,2.464568,0.005478,((((s4 ^ -0.32764328) - ((m2 - m7) / 1.4952106...,0.08607817**s1*(-0.668802107208175*m2 + 0.6688...,PySRFunction(X=>0.08607817**s1*(-0.66880210720...


In [ ]:
reg.equations_[0] = result

In [41]:
s = reg.latex_table(precision=2, columns=['equation', 'complexity', 'loss'])

# replace m_ with \\mu_ and s_ with \\sigma_
s = s.replace('m_', '\\mu_')
s = s.replace('s_', '\\sigma_')

# remove the first 5 lines which are package imports
s = '\n'.join(s.split('\n')[5:])
print(s)

\begin{table}[h]
\begin{center}
\begin{tabular}{@{}ccc@{}}
\toprule
Equation & Complexity & Loss \\
\midrule
$y_{0} = 7.0$ & $1$ & $5.7$ \\
$y_{0} = 7.0 - \mu_{2}$ & $3$ & $5.0$ \\
$y_{0} = 7.2 - \sin{\left(\mu_{2} \right)}$ & $4$ & $4.9$ \\
$y_{0} = \frac{3.6}{\sigma_{4}^{0.15}}$ & $5$ & $4.0$ \\
$y_{0} = - \mu_{2} + \frac{3.6}{\sigma_{4}^{0.16}}$ & $7$ & $3.3$ \\
$y_{0} = - \mu_{2} + \left(\sigma_{1} \sigma_{4}\right)^{-0.16} + 3.6$ & $9$ & $3.1$ \\
\begin{minipage}{0.8\linewidth} \vspace{-1em} \begin{dmath*} y_{0} = 0.14^{\sigma_{1}} \left(- \mu_{2} + \sigma_{4}^{-0.31}\right) + 3.7 \end{dmath*} \end{minipage} & $11$ & $2.7$ \\
\begin{minipage}{0.8\linewidth} \vspace{-1em} \begin{dmath*} y_{0} = 0.16^{\sigma_{1}} \left(- \mu_{2} + \sigma_{4}^{-0.32} - \sigma_{6}\right) + 3.7 \end{dmath*} \end{minipage} & $13$ & $2.6$ \\
\begin{minipage}{0.8\linewidth} \vspace{-1em} \begin{dmath*} y_{0} = 0.059^{\sigma_{1}} \left(\mu_{7} + \sigma_{4}^{-0.33} - \sin{\left(\mu_{2} \right)}\right) + 3.7

In [216]:
# now we can write it as a combination of the input features
# we'll sort the features by their absolute value to make it a bit easier to read
def feature_string(i, include_ssx=False, latex=False):
    transformation = linear_transformation(i)
    bias = input_bias[i]

    if include_ssx:
        expr = simplify_scaled_feature(transformation, bias)
        s = format_sympy_expr(expr, latex)
    else:
        s = format_transformation(transformation, bias, latex)

    # change + -'s to -'s
    s = s.replace(' + -', ' - ')
    return s


In [217]:
for i in range(input_linear.shape[0]):
    print(str(i) + ": " + feature_string(i, include_ssx=True, latex=False))

0: 14.0 * e1 - 35.0 * e2 + 0.911
1: 210 * a1 + 85400 * m1 - 211
2: 27000 * m1 - 4.02 * a3 + 6.01
3: 0.764 * sin_Omega3 - 1.10 * sin_Omega2 + 0.00289
4: 14.0 * a3 - 27.5 * a2 + 12.5
5: 29.1 * e3 + 1.47 * e1 - 1.34
6: 13.1 * i3 + 37.5 * i2 - 1.38
7: 58400 * m2 - 4.92 * e3 - 0.725
8: 21.1 * e1 - 4.48 * e3 - 0.948
9: 0.957 * e1 + 64600 * m3 - 0.839


In [218]:
def latex_string(include_ssx=False):
    s = ('\\begin{align*}\n'
        + f'ver&sion={version}\\\\'
        + 'f_1& \\text{ features:} \\\\ \n')

    for i in range(input_linear.shape[0]):
        s += f'    &{i}: {feature_string(i, include_ssx, latex=True)} \\\\ \n'

    s += '''\end{align*}'''
    return s


In [219]:
print(latex_string(include_ssx=True))

\begin{align*}
ver&sion=24880\\f_1& \text{ features:} \\ 
    &0: 14.0  e_1 - 35.0  e_2 + 0.911 \\ 
    &1: 210  a_1 + 85400  m_1 - 211 \\ 
    &2: 27000  m_1 - 4.02  a_3 + 6.01 \\ 
    &3: 0.764  \sin\Omega_3 - 1.10  \sin\Omega_2 + 0.00289 \\ 
    &4: 14.0  a_3 - 27.5  a_2 + 12.5 \\ 
    &5: 29.1  e_3 + 1.47  e_1 - 1.34 \\ 
    &6: 13.1  i_3 + 37.5  i_2 - 1.38 \\ 
    &7: 58400  m_2 - 4.92  e_3 - 0.725 \\ 
    &8: 21.1  e_1 - 4.48  e_3 - 0.948 \\ 
    &9: 0.957  e_1 + 64600  m_3 - 0.839 \\ 
\end{align*}


In [133]:
from IPython.display import display, Markdown
display(Markdown('$$\n' + latex_string(include_ssx=True) + '\n$$'))

$$
\begin{align*}
ver&sion=24880\\f_1& \text{ features:} \\ 
    &0: 14  e_1 - 35  e_2 + 0.91 \\ 
    &1: 2.1e+2  a_1 + 8.5e+4  m_1 - 2.1e+2 \\ 
    &2: 2.7e+4  m_1 - 4.0  a_3 + 6.0 \\ 
    &3: 0.76  \sin\Omega_3 - 1.1  \sin\Omega_2 + 0.0029 \\ 
    &4: 14  a_3 - 28  a_2 + 12 \\ 
    &5: 29  e_3 + 1.5  e_1 - 1.3 \\ 
    &6: 13  i_3 + 38  i_2 - 1.4 \\ 
    &7: 5.8e+4  m_2 - 4.9  e_3 - 0.72 \\ 
    &8: 21  e_1 - 4.5  e_3 - 0.95 \\ 
    &9: 0.96  e_1 + 6.5e+4  m_3 - 0.84 \\ 
\end{align*}
$$

In [42]:
version = 24880
model = spock_reg_model.load(version)
feature_nn = model.feature_nn

input_linear = feature_nn.linear.weight * feature_nn.mask
input_linear = input_linear.detach().numpy()
if feature_nn.linear.bias is not None:
    input_bias = feature_nn.linear.bias.detach().numpy()
else:
    input_bias = np.zeros(input_linear.shape[0])

In [43]:
def get_nonzero(arr):
    return arr[arr.nonzero()], arr.nonzero()

def set_nonzero(arr, values, indices):
    arr[indices] = values

In [44]:
def simplify(x, y, epsilon=0.1, zeroing_allowed=True):
    if x == 0 and y == 0:
        return (0, 0, 1), 0
    if x == 0:
        return (0, 1, 1/y), 0
    if y == 0:
        return (1, 0, 1/x), 0

    best_simplification, best_magnitude, best_error = None, None, None
    possible_values = list(range(-10, 11))
    candidate_ratios = [(a, c) for a in possible_values for c in possible_values]

    for a, b in candidate_ratios:
        if not zeroing_allowed and (a == 0 or b == 0):
            continue

        k = 1
        if a != 0:
            k = x / a
        if b != 0 and (a == 0 or abs(y) > abs(x)):
            k = y / b

        if k < 0: continue
        x2, y2 = k * a, k * b

        error = abs(x - x2) + abs(y - y2)
        # should be measured with the normalized values, not the original.
        if error >= epsilon: continue

        magnitude = abs(a) + abs(b)

        if best_error is None or magnitude < best_magnitude or magnitude == best_magnitude and error < best_error:
            best_simplification, best_error, best_magnitude = (a, b, x2, y2), error, magnitude

    return best_simplification, best_error

In [45]:
def simplify_stuff(input_linear, epsilon=0.1, normalize=True, zeroing_allowed=True):
    input_linear2 = input_linear.copy()
    for i in range(input_linear.shape[0]):
        nonzero, indices = get_nonzero(input_linear[i])
        x, y = nonzero
        print('original:\t', f'{x:.3f} {y:.3f}')

        if normalize:
            l1 = abs(x) + abs(y)
            x, y = x / l1, y / l1
            print('normalized:\t', f'{x:.3f} {y:.3f}')

        simplification, error = simplify(x, y, epsilon=epsilon, zeroing_allowed=zeroing_allowed)
        if simplification is None:
            print("no simplification found")
            nonzero = [x, y]
        else:
            a, b, x2, y2 = simplification
            print("new values:\t", f"{x2:.3f} {y2:.3f}", "with error", f"{error:.3f}")
            print("ratio:\t\t", f"{a} {b}")
            nonzero = [x2, y2]

            if normalize:
                l1 = abs(x2) + abs(y2)
                x2, y2 = x2 / l1, y2 / l1
                print("final normed:\t", f"{x2:.3f} {y2:.3f}")
                nonzero = [x2, y2]


        print()
        set_nonzero(input_linear2[i], nonzero, indices)

    input_linear2 = torch.tensor(input_linear2)
    feature_nn.linear.weight = torch.nn.Parameter(input_linear2)

    s = '24880_feature_nn_simplified_v3_'
    # if normalize:
        # s += 'norm_'
    # if not zeroing_allowed:
        # s += 'nozero_'
    s += f'eps={epsilon}.pt'

    torch.save(feature_nn, s)
    print(f'saved to', s)

In [46]:
# simplify_stuff(input_linear, epsilon=0.001, normalize=True, zeroing_allowed=False)

In [47]:
from sklearn.preprocessing import StandardScaler
import numpy as np
ssX = StandardScaler()
ssX.scale_ = np.array([2.88976974e+03, 6.10019661e-02, 4.03849732e-02, 4.81638693e+01,
           6.72583662e-02, 4.17939679e-02, 8.15995339e+00, 2.26871589e+01,
           4.73612029e-03, 7.09223721e-02, 3.06455099e-02, 7.10726478e-01,
           7.03392022e-01, 7.07873597e-01, 7.06030923e-01, 7.04728204e-01,
           7.09420909e-01, 1.90740659e-01, 4.75502285e-02, 2.77188320e-02,
           7.08891412e-01, 7.05214134e-01, 7.09786887e-01, 7.04371833e-01,
           7.04371110e-01, 7.09828420e-01, 3.33589977e-01, 5.20857790e-02,
           2.84763136e-02, 7.02210626e-01, 7.11815232e-01, 7.10512240e-01,
           7.03646004e-01, 7.08017286e-01, 7.06162814e-01, 2.12569430e-05,
           2.35019125e-05, 2.04211110e-05, 7.51048890e-02, 3.94254400e-01,
           7.11351099e-02])
ssX.mean_ = np.array([ 4.95458585e+03,  5.67411891e-02,  3.83176945e-02,  2.97223474e+00,
           6.29733979e-02,  3.50074471e-02,  6.72845676e-01,  9.92794768e+00,
           9.99628430e-01,  5.39591547e-02,  2.92795061e-02,  2.12480714e-03,
          -1.01500319e-02,  1.82667162e-02,  1.00813201e-02,  5.74404197e-03,
           6.86570242e-03,  1.25316320e+00,  4.76946516e-02,  2.71326280e-02,
           7.02054326e-03,  9.83378673e-03, -5.70616748e-03,  5.50782881e-03,
          -8.44213953e-04,  2.05958338e-03,  1.57866569e+00,  4.31476211e-02,
           2.73316392e-02,  1.05505555e-02,  1.03922250e-02,  7.36865006e-03,
          -6.00523246e-04,  6.53016990e-03, -1.72038113e-03,  1.24807860e-05,
           1.60314173e-05,  1.21732696e-05,  5.67292645e-03,  1.92488263e-01,
           5.08607199e-03])
ssX.var_ = ssX.scale_**2